In [ ]:
import evaluate

metric = evaluate.load("sacrebleu")

In [ ]:
datasets = ["all", "news", "news_sent_sampled/short", "news_sent_sampled/medium", "news_sent_sampled/long", "news_stroke/short", "news_stroke/medium", "news_stroke/long", "sent/short", "sent/medium", "sent/long", "stroke/short", "stroke/medium", "stroke/long"]
datasets = ["all"]

In [ ]:
src = "zh"
dataset = "simp"
for d in datasets:
    print(d)
    with open(f"../results/{dataset}/{d}/generate-test.hyp", 'r', encoding='utf-8') as f:
        preds = f.read().splitlines()
    
    splits = d.split("/")
    if len(splits) > 1:
        with open(f"../data/NIST/{dataset}/test/{splits[0]}/test-{splits[1]}.{src}-en.en", 'r', encoding='utf-8') as f:
            refs = f.read().splitlines()
    else:
        with open(f"../data/NIST/{dataset}/test/{d}/test.{src}-en.en", 'r', encoding='utf-8') as f:
            refs = f.read().splitlines()
    
    result = metric.compute(predictions=preds, references=refs)
    print(result)
    result = {"bleu": result["score"]}

In [ ]:
for d in datasets:
    print(d)
    with open(f"../results/trad/{d}/generate-test.hyp", 'r', encoding='utf-8') as f:
        trad_preds = f.read().splitlines()
    with open(f"../results/simp/{d}/generate-test.hyp", 'r', encoding='utf-8') as f:
        simp_preds = f.read().splitlines()
    
    splits = d.split("/")
    with open(f"../data/NIST/trad/test/{d}/test.tz-en.en", 'r', encoding='utf-8') as f:
        trad_refs = f.read().splitlines()
    with open(f"../data/NIST/simp_original/test/{d}/test.zh-en.en", 'r', encoding='utf-8') as f:
        simp_refs = f.read().splitlines()
    
    cnt = 0
    for i in range(len(trad_preds)):
        trad_result = metric.compute(predictions=[trad_preds[i]], references=[trad_refs[i]])
        simp_result = metric.compute(predictions=[simp_preds[i]], references=[simp_refs[i]])
        THRESHOLD = 15
        if simp_result["score"] - trad_result["score"] > THRESHOLD:
            # print(i)
            # print("simp better")
            print(simp_preds[i], '&', trad_preds[i], '&', trad_refs[i], '\\\\')
            cnt += 1
        elif simp_result["score"] - trad_result["score"] < -THRESHOLD:
            # print(i)
            print("trad better")
            print(simp_preds[i], '&', trad_preds[i], '&', trad_refs[i], '\\\\')
            cnt += 1
        if (cnt % 10 == 0):
            break